<a href="https://colab.research.google.com/github/msuraj248/DeepLearning/blob/master/LinearClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [2]:
data = pd.read_csv('Salary_data.csv')
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [17]:
data['race'].unique()

array([' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',
       ' Other'], dtype=object)

In [7]:
def lables(lable):
  if lable == '<=50K':
    return 0
  else:
    return 1

In [13]:
data['income_bracket'] = data['income_bracket'].apply(lables)
data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [ ]:
X = data.drop(['income_bracket'], axis = 1)
y = data['income_bracket']
y.head()

In [18]:
#Numerical_data
age = tf.feature_column.numeric_column('age')
edu_num = tf.feature_column.numeric_column('education_num')
cap_gain = tf.feature_column.numeric_column('capital_gain')
cap_loss = tf.feature_column.numeric_column('capital_loss')
time = tf.feature_column.numeric_column('hours_per_week')
#Categorical_data
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)
race = tf.feature_column.categorical_column_with_hash_bucket("race", hash_bucket_size=1000)
gender = tf.feature_column.categorical_column_with_hash_bucket("gender", hash_bucket_size=1000)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
feature_col = [age,edu_num,cap_gain,cap_loss,time,occupation,marital_status,relationship,education,workclass,native_country,race,gender]
model = tf.estimator.LinearClassifier(feature_columns=feature_col, n_classes=2)
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x = X_train,y = y_train,batch_size=10, num_epochs=1000, shuffle=True)
model.train(input_fn=input_func, steps=1000)

In [ ]:
eval_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x = X_test,y = y_test,batch_size=10, num_epochs=1, shuffle=False)
model.evaluate(eval_input_func)

In [35]:
pred_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x = X_test,batch_size=10, num_epochs=1, shuffle=False)
# pred_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(x= X_test, batch_size=len(X_test), shuffle=False)
result = list(model.predict(pred_input_func))
# prediction = list(model.predict(input_fn=pred_fun))
result[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpsjglci0j/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object),
 'logistic': array([1.], dtype=float32),
 'logits': array([17.167234], dtype=float32),
 'probabilities': array([3.5023927e-08, 1.0000000e+00], dtype=float32)}

In [36]:
# final_preds = []
# for pred in prediction:
#     final_preds.append(pred['class_ids'][0])

final_preds = []
for pred in result:
    final_preds.append(pred['class_ids'][0])

In [37]:
final_preds[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test,final_preds))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      9769

    accuracy                           1.00      9769
   macro avg       1.00      1.00      1.00      9769
weighted avg       1.00      1.00      1.00      9769

